## Exercise 3

### Descrição do Problema

O objetivo deste exercício é desenvolver uma AEAD com "Tweakable Block Ciphers”", 
sendo a cifra usada do tipo AES-256 ou o ChaCha20.

Esta cifra será usada num canal privade de informação assíncrona com acordo de chaves.
A autenticação dos agentes será com:
 - “X448 key exchange”
 - “Ed448 Signing&Verification” 
 
... e deve haver uma fase de confirmação da chave acordada.


Ou seja,

Use o “package” Cryptography para
1. Implementar uma AEAD com “Tweakable Block Ciphers” conforme está descrito na última secção do texto +Capítulo 1: Primitivas Criptográficas Básicas.  A cifra por blocos primitiva, usada para gerar a “tweakable block cipher”, é o AES-256 ou o ChaCha20.
2. Use esta cifra para construir um canal privado de informação assíncrona com acordo de chaves feito com “X448 key exchange” e “Ed448 Signing&Verification” para autenticação  dos agentes. Deve incluir uma fase de confirmação da chave acordada.

In [1]:
#Imports necessários
import os
import asyncio
import random
from pickle import dumps, loads
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import x448, ed448
from cryptography.hazmat.primitives.kdf.hkdf import HKDF
from cryptography.hazmat.primitives.asymmetric.x448 import X448PrivateKey
from cryptography.hazmat.primitives.asymmetric.ed448 import Ed448PrivateKey
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

# Para a porção de código associada com as queues (na comunicação entre participantes)
import nest_asyncio

nest_asyncio.apply()


import warnings
warnings.simplefilter("default") 

## Criação das chaves privadas/públicas

O acordo de chaves, como indicado no enunciado, deve ser feito com __"X448 key exchange"__.

Para tal é necessário que cada um dos agentes criem as suas próprias chaves (privada-pública).

Neste caso será usando a __curva448__, como 
apresentado na API:
- https://cryptography.io/en/latest/hazmat/primitives/asymmetric/x448/

A chave resultante será usada no decorrer da comunicação assíncrona entre agentes para __cifrar/decifrar__ os dados.

In [2]:
def generateKeys():
    # Generate private key for exchange
    prv_key = x448.X448PrivateKey.generate()
    
    # Generate public key thorugh private key
    pub_key = prv_key.public_key()
    
    return prv_key, pub_key
    

## Criação das chaves partilhadas

Após ter as chaves de cada agente (privada/pública), podemos estabelecer
o _exchange_, i.e o acordo entre ambos agentes,
sobre um segredo partilhado.

A _shared_key_ criada, tal como recomendado na API, deve ser passada por uma função de derivação, no âmbito de a tornar mais segura; adicionando mais informações à chave para destruir qualquer estrutura que possa ser criada.

Será criada uma _shared_key_, a ser usada no âmbito da cifra. 

In [3]:
def generateShared(prv_key, peer_key):
    
    peer_cipher_key = x448.X448PublicKey.from_public_bytes(peer_key)
    
    # Gerar uma chave partilha para cifra
    cipher_key = prv_key.exchange(peer_cipher_key)
    
    derived_key = HKDF(
        algorithm=hashes.SHA256(),
        length=16, #32,
        salt=None,
        info=b'handshake data',
    ).derive(cipher_key)
    
    return derived_key

## Assinar mensagem

A autenticação dos agentes é realizada com 
__“Ed448 Signing&Verification”__; sendo este um 
algoritmo de assinaturas com recurso ao __EdDSA__ (_Edwards-curve Digital Signature Algorithm_).

Para tal é necessário um novo par de chaves (privada/pública), do tipo _Ed448_, para cada um dos agentes.

Para recorrer aos meios apresentados em:
- https://cryptography.io/en/latest/hazmat/primitives/asymmetric/ed448/

As chaves públicas devem ser partilhadas por ambos, de 
modo a validar a __assinatura__ da mensagem que receberam.
Sendo a assinatura realizada através da chave privada do agente,
a enviar a mensagem.

Criou-se um método particular para gerar as chaves de autenticação dos agentes, para facilitar e tornar o código mais _readable_.

In [4]:
def generateSignKeys():
    
    ## Chave privada para assinar
    private_key = Ed448PrivateKey.generate()
    
    ## Chave pública para autenticar
    public_key = private_key.public_key()
   
    return private_key, public_key
    
def signMsg(prv_key, msg):
    
    signature = prv_key.sign(msg)
    
    return signature

## Funções auxiliares para cifragem

Os três métodos auxiliares criados, são no âmbito de:
- Adicionar padding ao _plaintext_, sendo dado o tamanho de cada bloco.
... Obtém-se o último bloco do _plaintext_, e 
preenche os restantes _bytes_ (da _lenght_ que cada bloco deve ter) 
com __0__ (neste caso _\x00_).
Retorna-se o _plaintext_ atualizado e o último bloco (_padded_).


In [5]:
def pad (block_plaintext, length):
    last_block = block_plaintext[-1] # Get the last block of the list
    len_last_block = len(last_block)
    
    for _ in range (len_last_block, length): # Adds the value 0 until the size the last block is 0
        last_block += b"\x00"
    
    block_plaintext[-1] = last_block
    
    return block_plaintext, len_last_block

    
def tpbc (tweak, key, block, iv):
    tweaked_key = tweak + key
    cipher = Cipher(algorithms.AES256(tweaked_key), modes.CBC(iv))
    encryptor = cipher.encryptor()
    ct_block = encryptor.update(block) + encryptor.finalize()
    return ct_block

# nounce: bytes, ctr: bytes, block_plaintext: bytes, key: bytes, auth: bytes
def tweakable_first_blocks(nounce, ctr, block_plaintext, key, auth, iv):
    ct = b""
    zero = b"\x00"
    
    for elem in (block_plaintext[:-1]):        
        tweak = nounce + ctr + zero
        
        c_i = tpbc(tweak, key, elem, iv)
        ct += c_i
        
        len_ctr = len(ctr)
        ctr_int = int.from_bytes(ctr, 'big')
        ctr_int += 1
        ctr = ctr_int.to_bytes(len_ctr, 'big')
        
        aux = b""
        for x,y in zip(auth, elem):
            word = x ^ y
            aux += word.to_bytes(1, 'big')
        
        auth = aux
        
    return ctr, auth, ct

## Cifrar mensagem

In [6]:
def encrypt(msg, key):
    NBytes = 16; size_nounce = 8
    
    digest = hashes.Hash(hashes.SHA3_256())
    nounce_temp = digest.finalize()
    nounce = nounce_temp[:size_nounce] # array of bytes with size 8
    
    ctr_i = os.urandom(size_nounce-1) # array of bytes with size 7
    ctr = ctr_i 
    
    iv = os.urandom(16)
        
    block_plaintext = [bytes(msg[i:i+NBytes], 'utf8') for i in range(0, len(msg), NBytes)]  # List of block of bytes. Block size 16
    
    block_plaintext, len_last_block = pad(block_plaintext, NBytes) # blocks de bytes de tam 16, tamanho original do ultimo bloco 
    
    # i = 0 ... m - 1    
    auth = b"" 
    for _ in range (NBytes): # array of bytes with size 16 bytes
        auth += b"\x00"

    ctr, auth, ct = tweakable_first_blocks(nounce, ctr, block_plaintext, key, auth, iv) # ctr: bytes; auth: str ; ct: bytes

    # i = m
    tweak = nounce + ctr +  b"\x00"
    length_block = len_last_block.to_bytes(16, 'big') # Turns the length of the last block into a 16 bytes block
    c_aux = tpbc(tweak, key, length_block, iv) # c_aux: bytes
    
    c_m = b""
    for x,y in zip(block_plaintext[-1], c_aux):
        word = x ^ y
        c_m += word.to_bytes(1, 'big')
    ct += c_m # ct: bytes
    
    
    aux = b""
    for x,y in zip(auth, block_plaintext[-1],):
        word = x ^ y
        aux += word.to_bytes(1, 'big')
    auth = aux
    
    # Autenticação
    tweak = nounce + ctr + b"\x01"

    tag = tpbc(tweak, key, auth, iv)
    
    return {"ct": ct, "tag": tag, "nounce": nounce, "ctr": ctr_i, "pad": len_last_block, "iv": iv}

## Funções auxiliares para decifragem

In [7]:
# size_block = tau
def unpad(last_block, size_block, NBytes):
    
    last_block = last_block[:size_block]
    
    return last_block


def un_tpbc(tweak, key, block, iv):
    tweaked_key = tweak + key
    cipher = Cipher(algorithms.AES256(tweaked_key), modes.CBC(iv))
    decryptor = cipher.decryptor()
    plain_block = decryptor.update(block) + decryptor.finalize()
    return plain_block

def undo_tweakable_first_blocks(nounce, ctr, block_ciphertext, key, auth, iv):
    plaintext = b""    
    for elem in (block_ciphertext[:-1]):        
        tweak = nounce + ctr +  b"\x00"
        
        c_i = un_tpbc(tweak, key, elem, iv)
        plaintext += c_i
        
        len_ctr = len(ctr)
        ctr_int = int.from_bytes(ctr, 'big')
        ctr_int += 1
        ctr = ctr_int.to_bytes(len_ctr, 'big')
        
        aux = b""
        for x,y in zip(auth, c_i):
            word = x ^ y
            aux += word.to_bytes(1, 'big')
        
        auth = aux
        
    return ctr, auth, plaintext



## Decifrar mensagem

In [8]:
def decrypt(msg, key):
    ct = msg['ct'] # bytes
    tag_rcv = msg['tag']
    nounce = msg['nounce']
    ctr = msg['ctr']
    len_last_block = msg['pad']
    iv = msg['iv']
    
    NBytes = 16
    
    block_ciphertext = [ct[i:i+NBytes] for i in range(0, len(ct), NBytes)]  # list of block of bytes. Block size 16 bytes
    
    # i= 0 ... m - 1
    auth = b"" 
    for _ in range (NBytes): # array of bytes with size 16 bytes
        auth += b"\x00"
    ctr, auth, plaintext = undo_tweakable_first_blocks(nounce, ctr, block_ciphertext, key, auth, iv)
        
    # i = m
    tweak = nounce + ctr + b"\x00"
    length_block = len_last_block.to_bytes(16, 'big')
    c_aux = tpbc(tweak, key, length_block, iv)
    
    c_m = b""
    for x,y in zip(block_ciphertext[-1], c_aux):
        word = x ^ y
        c_m += word.to_bytes(1, 'big')
    
    plaintext += c_m[:len_last_block] # ct: bytes
    
    aux = b""
    for x,y in zip(auth, c_m):
        word = x ^ y
        aux += word.to_bytes(1, 'big')
    auth = aux
    
    # Autenticação
    tweak = nounce + ctr + b"\x01"
    
    tag = tpbc(tweak, key, auth, iv)
    
    
    if tag != tag_rcv:
        print("Mensagem inválida")
    else:
        print("Mensagem válida")
       
    return plaintext

## Funções auxiliares para o canal de comunicação
Encontra-se 3 métodos auxiliares para:
- inicialização dos agentes;
- Envio de mensagens;
- Receção de mensagem.

Os métodos para envio e receção de mensagens
têm em consideração o código proporcionado
pela equipa docente, em consideração das 
estruturas _queues_.

O método para __inicialização__:
- Cria os dois pares de chaves pública-privada (para cifrar e assinar);
- Serializa as chaves públicas (para enviar ao _peer_);
- Retorna um tuplo com as chaves e o pacote a enviar ao peer.

...com o nome _init_comm().

É realizada uma __assinatura da chave pública para cifrar__ à priori, correpondendo
à fase de __confirmação da chave acordada__.
Uma vez que esta é uma chave temporária, e usada unicamente para confirmar que a assinatura
está correta, não inclui tantas informações para a tornar mais crédula; por exemplo
certificados, protocolos, etc...

Sendo assim, a fase de confirmação desta chave de __curta duração__ é, essencialmente,
a criação de uma assinatura para a avaliar ao ser enviada ao respetivo _peer_.

In [9]:
def init_comm():
    ## Gerar chaves (cifrar e autenticar)
    prv_cipher_key, pub_cipher_key = generateKeys()
    
    prv_sign_key, pub_sign_key = generateSignKeys()
    
    msg_to_sign = pub_cipher_key.public_bytes(encoding=serialization.Encoding.Raw,
                                       format=serialization.PublicFormat.Raw
                                    )
    
    ##Assinar a public cipher key - autenticação da chave acordada
    sign = signMsg(prv_sign_key, msg_to_sign)
    
    ## Dicionário com a chavess públicas (serializadas)
    msg = {'cipher_key': pub_cipher_key.public_bytes(encoding=serialization.Encoding.Raw,
                                       format=serialization.PublicFormat.Raw
                                    ), 
           'sign_key': pub_sign_key.public_bytes(encoding=serialization.Encoding.Raw,
                                       format=serialization.PublicFormat.Raw
                                    ),
           'sig': sign
            }
    
    return prv_cipher_key, prv_sign_key, msg

async def send(queue, msg):
    
    await asyncio.sleep(random.random())
        
    # put the item in the queue
    await queue.put(msg)
    
    await asyncio.sleep(random.random())
    
async def receive(queue):
    item = await queue.get()

    await asyncio.sleep(random.random())
    aux = loads(item)

    return aux

## Emitter

O processo do _emitter_ passa pelos passos:

- Criar dois pares de chaves privadas/públicas (para __cifrar__ e __assinar__);
- Assinar a chave pública de cifragem (com a chave privada de assinar);
- Enviar as chaves públicas (de cifragem e para verificar a assinatura);
- Receber as respetivas chaves públicas do peer (e a assinatura);
- Validar a assinatura;
- Gerar o segredo partilhado para cifragem (chave partilhada - _cipher_)
- Cifrar a mensagem com a chave partilhada (_encrypt_)
- Assinar com a chave privada (para assinar)
- Enviar pacote para o _receiver_

In [10]:
## Emitter Code
async def emitter(plaintext, queue):
    
    ## Gerar as chaves (privada e publica) & partilhar com participante
    prv_cipher_key, prv_sign_key, msg = init_comm()
    
    ## Enviar a chaves públicas para o peer
    print("[E] SENDING PUBLIC KEYS")
    await send(queue, dumps(msg))
    
    ## Receber as chaves públicas do peer
    msg = await receive(queue)
    print("[E] RECEIVED PEER PUBLIC KEYS")
    
    pub_peer_cipher = msg['cipher_key']
    pub_peer_sign = msg['sign_key']
    signature = msg['sig']
    print("[E] Receiver pub_key_cipher: " +str(pub_peer_cipher))
    print("[E] Receiver pub_key_sign: " +str(pub_peer_sign))
    print("[E] Receiver signature: " +str(signature))
    
    try:
        ## Obter a chave pública (Assinatura)
        peer_sign_pubkey = ed448.Ed448PublicKey.from_public_bytes(pub_peer_sign)
        
        ## Verificar a assinatura da chave pública
        peer_sign_pubkey.verify(signature, pub_peer_cipher)
        print("[E] SIGNATURE VALIDATED")
        
        ## Criar as chaves partilhadas (cifrar/autenticar)
        cipher_shared = generateShared(prv_cipher_key, pub_peer_cipher)

        print("[E] CIPHER SHARED: "+str(cipher_shared))

        ## Cifrar a mensagem
        pkg = encrypt(plaintext, cipher_shared)
        print("[E] MESSAGE ENCRYPTED")

        ## Assinar e enviar a mensagem
        pkg_b = dumps(pkg)
        sig = signMsg(prv_sign_key, pkg_b)

        ## a Enviar...
        msg_final = {'sig': sig, 'msg': dumps(pkg)}

        print("[E] SENDING MESSAGE")
        await send(queue, dumps(msg_final))

        print("[E] END")
        
    except InvalidSignature:
        printf("A assinatura não foi verificada com sucesso!")

## Receiver

Os dois agentes, no contexto deste exemplo, são particularmente 
semelhantes, apenas só alterando a ordem dos eventos. Neste caso,
é do _emitter_ para o _receiver_.

Sendo assim, os eventos surgem como:
- Criar dois pares de chaves privadas/públicas (para cifrar e assinar);
- Receber as respetivas chaves públicas do peer (e a assinatura);
- Validar a assinatura;
- Gerar o segredo partilhado para cifragem (chave partilhada - _cipher_);
- Enviar as chaves públicas (de cifragem e para verificar a assinatura);
- Receber o criptograma do _emitter_;
- Validar a assinatura do criptograma;
- Decifrar o pacote e obter o _plain_text_;
- Imprimir o _plaintext_.

In [11]:
## Receiver Code
async def receiver(queue):
    
    ## Gerar as chaves (privada e publica) & partilhar com participante
    prv_cipher_key, prv_sign_key, msg = init_comm()
    
    ## Receber as chaves publicas do peer
    pub_keys = await receive(queue)
    
    pub_peer_cipher = pub_keys['cipher_key']
    pub_peer_sign = pub_keys['sign_key']
    signature = pub_keys['sig']
    print("[R] Emitter pub_key_cipher: " +str(pub_peer_cipher))
    print("[R] Emitter pub_key_sign: " +str(pub_peer_sign))
    print("[R] Receiver signature: " +str(signature))
    
    try:
        ## Obter a chave pública (Assinatura)
        peer_sign_pubkey = ed448.Ed448PublicKey.from_public_bytes(pub_peer_sign)
        
        ## Validar a correçaõ da assinatura
        peer_sign_pubkey.verify(signature, pub_peer_cipher)
        print("[R] SIGNATURE VALIDATED")
        
        ## Gerar shared keys
        cipher_shared = generateShared(prv_cipher_key, pub_peer_cipher)

        ## Enviar as chaves públicas ao peer
        print("[R] SEND PUBLIC KEYS")
        await send(queue, dumps(msg))
        
        ## Receber criptograma
        print("[R] AWAIT CIPHER")
        ciphertext = await receive(queue)
        print("[R] CIPHER RECEIVED")

        try:     
            ## Validar a correção da assinatura
            peer_sign_pubkey.verify(ciphertext['sig'], ciphertext['msg'])
            print("[R] SIGNATURE VALIDATED")

            msg_dict = loads(ciphertext['msg'])

            ## Decifrar essa mensagem       
            plain_text = decrypt(msg_dict, cipher_shared)
            print("[R] MESSAGE DECRYPTED")

            ## Apresentar no terminal
            print("[R] PLAINTEXT: " + plain_text.decode('utf-8'))

            print("[R] END")

        except InvalidSignature:
            print("The signature wasn't validated correctly! - Cipher")
            
    except InvalidSignature:
        print("The signature wasn't validated correctly! - Cipher key")

## Teste de execução

Encontra-se na célula abaixo um exemplo de execução
da comunicação entre um _emitter_ e um _receiver_,
acompanhada das respetivas autenticações do agentes, da mensagem,
bem como a cifragem/decifragem da última.

O método:
```python
def test(msg)
```

... funciona como um método _main_ para a execução; recebendo
como argumento a __mensagem__.

In [12]:
def test(msg):
    loop = asyncio.get_event_loop()
    queue = asyncio.Queue(10)
    asyncio.ensure_future(emitter(msg, queue), loop=loop)
    loop.run_until_complete(receiver(queue))
    
test("HELLO WORLD! THIS IS JUST A TEST OF A AEAD ALGORITHM.")

[E] SENDING PUBLIC KEYS
[R] Emitter pub_key_cipher: b'\x96\x98\x9d9\xc40c2\xd6\xcb\xdau#\xf8\xe0\xc1X\xd9>\xdb\xe7\x90\x7fQ\xdb\xaf.\xfd\x95t\xc6\xab\xfd\x05q\xc5\x1c\x9e(4\xa9\x02\r\x82j\xc9=\xa6D)\x00\x9f5\x11\x9aZ'
[R] Emitter pub_key_sign: b"hi \xad\x04\xf8\xf8\xf0\xa7v\x88\xde\x90i\xbf`S\x9e5\x85\xadL\xcc\xc6]\xf7\x88'9@\xaa\x88\xcb\xd8]\x0f]\xc6\xb7\xef_w\x91z\xa7e\xcepH \x8f\x99\xb2\x8e\\n\x80"
[R] Receiver signature: b'D_\x8f\xcb\x0c\x9c1\xd4\xd9Me\xe9\x7f4\xf1 z|;i\xfa\xfe\xdf\x06E\x98\xca\x03<f\xf6\x87y\x0c\x9c\x8f\xa0UU\xb4\x9b\xa6n\xad\xa0R86\xda\xc1X\xc74\x81\xbco\x00p\xaa>,@\xda~\xde\x05\x19\xb8XG\x80\x0b\x9d\xbcR\xa5&\x98(2\xc5\x99zZ\x86\x96\xd3\xc2\x0fW\xa6\xbc\xae\x19\xfa\xe6#-\xf6p-?d\xf3\xeeQ\xe6\xaf\xc9\xe9\x8b\xeb)\x00'
[R] SIGNATURE VALIDATED
[R] SEND PUBLIC KEYS
[R] AWAIT CIPHER
[E] RECEIVED PEER PUBLIC KEYS
[E] Receiver pub_key_cipher: b"\x040Jl\x93\x1a\xc9\xa9\xcbvT\xbe\xa6ct~\xc9\x15\x1dEXT\xe8\x8b\xf5\x16\xd4?\xba2x\xd7\xaf\xfeD\x91\x869\xb3?\x9a; \xf6'\x9a;\

## Outro teste de execução

Neste é apresentada a possibilidade de se inserir um _input_ do utilizador,
como mensagem a ser enviada do _emitter_ para o _receiver_, 
a passar pela cifra.

In [13]:
msg = input()

test(msg)

 HELLO WORLD


[E] SENDING PUBLIC KEYS
[R] Emitter pub_key_cipher: b"\x19\xcaZL\x9cF\xf4\x02\xc0\x8f\xc5\x0bZ\xb5\xec\r=\x8dfs\x83\x91\x85g\x10\xd3\x82\xd8\xef\x13\x9a\xe5yu\n\xb1\x8d18\x7fA\xbe\x9f\xf6\x1f\xbbc\xe0\xae\xb9\x14j'\xb6\xf2\xe4"
[R] Emitter pub_key_sign: b"\xa6j}^\xe9\x0f\xe7\xd0\x90%\xd0\x1fe7J\x14\xea!;\xf1\xf4\xe62a\xcel\xad\xab\xf1\xd6Y\xd6_\xaa\xe5x|N\x18\x9d\x0b\xect\xcb'!\xa4\xd3\xfc4E=\x8a\\v\x86\x80"
[R] Receiver signature: b'\xb5\xb05\x00\x9b\xef<\xca\x14\xf4\xc8i\x01\xa9\x8d\xef\xabl\xb1\x95\x97\xfcu\xee\x9b\xe6{\xacJj\x8f[\xc2h\x16\xb3\xa5uS\tQ\xa6\xa9\xa2}\xb4\xaa\xf0\xa9\xc4sZ\xcad\xd5\x8e\x00\x1d\x02\xdc>\xd4\x13\xd5\x08\x15\xbb\x94\x11\xc7\x7f\x01#u,\xea;\xbfQ\xb3\xb9\x81\x0c\xa2s8s\xb7\xd8\xcf?\xf8\x81]\xf4\x05z\xc2\xa4f\x9a\x86nRN\xe680\x1f\x8e\x87\t0\x00'
[R] SIGNATURE VALIDATED
[R] SEND PUBLIC KEYS
[E] RECEIVED PEER PUBLIC KEYS
[E] Receiver pub_key_cipher: b"\xc86hXd0_e\x84\xd9IYi\xbb\xf7\xab#W\x05\x8c\xad\xd6'\xbc\x9b\x10\xe7\xb6\xfe\xa2\xc7\xe9\xc54\xdc>\x94\x18V\x